In [ ]:
import os
import igraph # I dont think this package is on google colab, might be better to use NetworkX
import pandas as pd
from tqdm import tqdm_notebook as tqdm


Read dataframe with metadata.

In [ ]:
df = pd.read_csv('../Data/MitoTable.csv') 
df.Type = df.Type.astype("category")
df.Type.cat.reorder_categories(["WT", "WTFIX",'DD', 'DDFIX'], inplace=True)
print(df.shape)

df = df.loc[(df.Date.isin([24]))&(df.Type.isin(['WT','DDFIX']))]
df.Type = df.Type.cat.remove_unused_categories()
print(df.shape)

Read individual steady-state MitoGraphs:

In [ ]:
for index in tqdm(df.index):
        
    # Load the graph
    fname = os.path.join('../Data/',df.Name[index]+'.gnet') 
    graph_list = pd.read_csv(fname, skiprows=1, sep='\t', header=None)
    graph_list.columns = ['Source','Target','Length']
    edges = [(i,j) for i,j in zip(graph_list.Source.values,graph_list.Target.values)]
   
    g = igraph.Graph(edges=edges, directed=False) # this is the variable that stores the mitochondria graph
    
    
    # Other optional data we can load: edge length, cell size
    g.es['length'] = graph_list.Length.values 

    fname = os.path.join('../Data/',df.Name[index].replace('_fix','')+'.mbcellvol') 
    cell_size = pd.read_csv(fname, skiprows=1, sep='\t', header=None)
    cell_size.columns = ['MinorRadius','MidRadius','MajorAxis','SurfaceArea','Volume']
